In [27]:
# For Google Colaboratory
import sys, os
if 'google.colab' in sys.modules:
    from google.colab import drive
    drive.mount('/content/gdrive')
    file_name = 'conv1d.ipynb'
    import subprocess
    path_to_file = subprocess.check_output('find . -type f -name ' + str(file_name), shell=True).decode("utf-8")
    print(path_to_file)
    path_to_file = path_to_file.replace(file_name,"").replace('\n',"")
    os.chdir(path_to_file)
    !pwd4/rwG2zPYz5nLw4oxBo_eyUAEiKbHKef-aAv0iLRHPbfc_HJc1tMyuWYo

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
./conv1d.ipynb

/bin/bash: pwd4/rwG2zPYz5nLw4oxBo_eyUAEiKbHKef-aAv0iLRHPbfc_HJc1tMyuWYo: No such file or directory


In [0]:
import torch
import torch.nn as nn
import numpy as np
import time
import torch.nn.functional as F
import random
from matplotlib import pyplot as plt

In [29]:
trainer1=np.load("trainData__SMOTE_all_10s_f0.npz","rb")
trainer2=np.load("trainData__SMOTE_all_10s_f0_TEST.npz", "rb") 

x_train=torch.from_numpy(trainer1['x'][:50000])
y_train=torch.from_numpy(trainer1['y'][:50000])
y_train=y_train.long()

x_test=torch.from_numpy(trainer1['x'][50000:])
y_test=torch.from_numpy(trainer1['y'][50000:])
y_test=y_test.long()

print("Training size= ",y_test.type())
print("Testing size= ", x_test.size())

Training size=  torch.LongTensor
Testing size=  torch.Size([3430, 3000])


In [30]:
std= x_train.std()
mean=x_train.mean()

print(std,mean,sep="  ")

tensor(0.9725)  tensor(-1.4974e-10)


In [0]:
batch_size = 100
learning_rate = 0.01

class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet,self).__init__()
        
        #Pool and activation fn's here 
        activation=nn.ReLU()       
        pool_8 = nn.MaxPool1d(kernel_size=8,stride=8) #Size=8
        pool_4 = nn.MaxPool1d(kernel_size=4,stride=4) #Size=4
        pool_2 = nn.MaxPool1d(kernel_size=2,stride=2) #Size=2
        
        #Dropout
        self.drop_out = nn.Dropout()
        
        
        #ConvNet time
        conv1_t=nn.Conv1d (1, 64, kernel_size=50,stride=6,padding=1)
        self.conv2_t=nn.Conv1d(64,128,kernel_size=8,stride=1,padding=1)
        self.conv3_t=nn.Conv1d(128,128,kernel_size=8,stride=1,padding=1)
        conv4_t=nn.Conv1d(128,128,kernel_size=8,stride=1,padding=1)
        
        #Layers time
        self.layer1_t=nn.Sequential(conv1_t,activation,pool_8)
        self.layer2_t=nn.Sequential(conv4_t,activation,pool_4)
        
        #ConvNet freq
        
        conv1_f=nn.Conv1d(1,  64,  kernel_size=400, stride=50,padding=1)
        self.conv2_f=nn.Conv1d(64, 128, kernel_size=6,   stride=1,padding=1)
        self.conv3_f=nn.Conv1d(128,128, kernel_size=6,   stride=1,padding=1)
        conv4_f=nn.Conv1d(128,128, kernel_size=6,   stride=1,padding=1)
        
        
        #Layers freq
        
        self.layer1_f=nn.Sequential(conv1_f, activation, pool_4)
        self.layer2_f=nn.Sequential(conv4_f, activation, pool_2)
        
        self.layer=nn.Sequential(nn.Linear(1664,5,bias=False),activation)
        
        
        
        
   
    def forward(self,x):
        
        
        #Time here
        out_t=self.layer1_t(x)
        out_t=self.drop_out(out_t)  #DropOut
        out_t=self.conv2_t(out_t)
        out_t=self.conv3_t(out_t)
        out_t=self.layer2_t(out_t)
        
        #Freq here
        
        out_f=self.layer1_f(x)
        out_f=self.drop_out(out_f)  #DropOut
        out_f=self.conv2_f(out_f)
        out_f=self.conv3_f(out_f)
        out_f=self.layer2_f(out_f)
        
        out=torch.cat((out_t,out_f),2)  #Concat
        out=self.drop_out(out)   #DropOut
        out=out.view(-1,1664)
        
        out=self.layer(out)
        
        #print("Output size=  ",size)
       # print("modified= ",out.reshape(-1) )
        return out

In [32]:
net=ConvNet()
print(net)
net = net.to(torch.device("cuda"))

mean=mean.to(torch.device("cuda"))

std=std.to(torch.device("cuda"))
criterion = nn.CrossEntropyLoss()

ConvNet(
  (drop_out): Dropout(p=0.5, inplace=False)
  (conv2_t): Conv1d(64, 128, kernel_size=(8,), stride=(1,), padding=(1,))
  (conv3_t): Conv1d(128, 128, kernel_size=(8,), stride=(1,), padding=(1,))
  (layer1_t): Sequential(
    (0): Conv1d(1, 64, kernel_size=(50,), stride=(6,), padding=(1,))
    (1): ReLU()
    (2): MaxPool1d(kernel_size=8, stride=8, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2_t): Sequential(
    (0): Conv1d(128, 128, kernel_size=(8,), stride=(1,), padding=(1,))
    (1): ReLU()
    (2): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2_f): Conv1d(64, 128, kernel_size=(6,), stride=(1,), padding=(1,))
  (conv3_f): Conv1d(128, 128, kernel_size=(6,), stride=(1,), padding=(1,))
  (layer1_f): Sequential(
    (0): Conv1d(1, 64, kernel_size=(400,), stride=(50,), padding=(1,))
    (1): ReLU()
    (2): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2_f): Sequential(
    (0): Conv1d(128, 1

In [0]:
def get_error(scores,labels):
  
  batch_size = scores.size(0)
  predicted_labels = scores.argmax(dim=1)
  indicator = (predicted_labels == labels)
  num_matches=indicator.sum()
  
  return 1-num_matches.float()/batch_size

In [0]:
def eval_on_test_set():

    running_error=0
    num_batches=0

    for i in range(0,1000,batch_size):

        minibatch_data =  x_test[i:i+batch_size].unsqueeze(dim=1)
        minibatch_label= y_test[i:i+batch_size]

        minibatch_data=minibatch_data.to(torch.device("cuda"))
        minibatch_label=minibatch_label.to(torch.device("cuda"))
        
        inputs = (minibatch_data - mean)/std    # ONLY CHANGE IS HERE!

        scores=net( inputs ) 

        error = get_error( scores , minibatch_label)

        running_error += error.item()

        num_batches+=1


    total_error = running_error/num_batches
    print( 'error rate on test set =', total_error*100 ,'percent')

In [35]:
start=time.time()
epoch_axis=list()
error_axis=list()

for epoch in range(1,20):
    
       
    optimizer=torch.optim.SGD( net.parameters() , lr=learning_rate )
        
    running_loss=0
    running_error=0
    num_batches=0
    
    shuffled_indices=torch.randperm(40000)
 
    for count in range(0,10000,batch_size):
        
        # FORWARD AND BACKWARD PASS
    
        optimizer.zero_grad()
             
        indices=shuffled_indices[count:count+batch_size]
        minibatch_data =  x_train[indices].unsqueeze(dim=1)
        minibatch_label=  y_train[indices]
        
        #print("minibatch_data= ",minibatch_data.size())
        #print("minibatch_label= ",minibatch_label.size())
        
        
        minibatch_data=minibatch_data.to(torch.device("cuda"))
        minibatch_label=minibatch_label.to(torch.device("cuda"))
        
        inputs = (minibatch_data - mean)/std      # ONLY CHANGE IS HERE!
        
        inputs.requires_grad_()
                
        scores=net( inputs ) 
        #print("SCores=", scores.size())

        loss =  criterion( scores , minibatch_label) 
          
        loss.backward()
        
        optimizer.step()
        

        # COMPUTE STATS
        
        running_loss += loss.detach().item()
        
        error = get_error( scores.detach() , minibatch_label)
        running_error += error.item()
                   
        
        
        
        num_batches+=1        
    
    
    # AVERAGE STATS THEN DISPLAY
    total_loss = running_loss/num_batches
    total_error = running_error/num_batches
    elapsed = (time.time()-start)/60
    
    print('epoch=',epoch, '\t time=', elapsed,'min', '\t learning rate=', learning_rate  ,'\t loss=', total_loss , '\t error=', total_error*100 ,'percent')
    epoch_axis.append(epoch)
    error_axis.append(error)
    eval_on_test_set() 
    print(' ')

#plt.plot(epoch_axis,error_axis)
#plt.show()

epoch= 1 	 time= 0.037482929229736325 min 	 learning rate= 0.01 	 loss= 1.4811753690242768 	 error= 60.200001060962684 percent
error rate on test set = 100.0 percent
 
epoch= 2 	 time= 0.07266821066538492 min 	 learning rate= 0.01 	 loss= 1.446482824087143 	 error= 59.0900011062622 percent
error rate on test set = 100.0 percent
 
epoch= 3 	 time= 0.10795209407806397 min 	 learning rate= 0.01 	 loss= 1.3931188082695007 	 error= 54.64000141620636 percent
error rate on test set = 100.0 percent
 
epoch= 4 	 time= 0.14305208126703897 min 	 learning rate= 0.01 	 loss= 1.3147796523571014 	 error= 51.15000104904175 percent
error rate on test set = 98.40000092983246 percent
 
epoch= 5 	 time= 0.17808174689610798 min 	 learning rate= 0.01 	 loss= 1.1444898915290833 	 error= 44.76000118255615 percent
error rate on test set = 49.20000076293945 percent
 
epoch= 6 	 time= 0.2132912238438924 min 	 learning rate= 0.01 	 loss= 1.0298060393333435 	 error= 39.1100013256073 percent
error rate on test set 

In [39]:
#Tester

test_no = random.randint(0,1)
test_data = x_test[test_no]
test_data = test_data.view(1,3000).unsqueeze(dim=1)
test_label= y_test[test_no]

test_data = test_data.to(torch.device("cuda"))
test_data = (test_data-mean)/std

#print(test_data.size())
scores = net(test_data)
#print(scores)
prob = F.softmax(scores, dim=1)

##index



print(test_label)
print(prob)


tensor(3)
tensor([[0.0191, 0.0043, 0.1092, 0.8613, 0.0062]], device='cuda:0',
       grad_fn=<SoftmaxBackward>)
